In [2]:
import PySimpleGUI as sg
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import itertools

def generate_pdf(contract, cost, results):
    filename = "profit_report.pdf"
    c = canvas.Canvas(filename, pagesize=letter)
    c.drawString(100, 750, "Profit Possibilities Report")

    c.drawString(100, 700, f"Contract Value: {contract}")
    c.drawString(100, 680, f"Cost Value: {cost}")
    c.drawString(100, 660, "")

    y_offset = 620
    for combination, gross_profit_percentage in results:
        factors_str = ', '.join(combination) if combination else 'No factors'
        c.drawString(100, y_offset, f"Factors: {factors_str} => Gross Profit %: {gross_profit_percentage:.2f}%")
        y_offset -= 20

    c.save()
    sg.popup(f"PDF report generated: {filename}")
    sg.PopupOK(f"PDF report generated: {filename}")

def calculate_profit_percentage(contract, cost, factors):
    factor_combinations = []
    for r in range(0, len(factors) + 1):
        factor_combinations.extend(list(itertools.combinations(factors.keys(), r)))

    results = []
    for combination in factor_combinations:
        combined_impact = 0
        for factor in combination:
            impact_on, impact_value = factors[factor]
            if impact_on.lower() == 'c':
                cost += impact_value
            elif impact_on.lower() == 't':
                contract += impact_value
            combined_impact += impact_value

        gross_profit = contract - cost
        gross_profit_percentage = (gross_profit / contract) * 100
        results.append((combination, gross_profit_percentage))
        contract -= combined_impact if combined_impact else 0
        cost -= combined_impact if combined_impact else 0

    return contract, cost, results

def main():
    layout_basic_info = [
        [sg.Text('Contract Value'), sg.InputText(key='contract')],
        [sg.Text('Cost of the Project'), sg.InputText(key='cost')],
        [sg.Text('Number of Factors'), sg.InputText(key='num_factors')],
        [sg.Button('Next')]
    ]

    window_basic_info = sg.Window('Basic Information').Layout(layout_basic_info)
    while True:
        event, values = window_basic_info.Read()
        if event == 'Next':
            contract = float(values['contract'])
            cost = float(values['cost'])
            num_factors = int(values['num_factors'])
            break
        if event is None:
            break

    window_basic_info.Close()

    layout_factors = [
        [sg.Text(f'Factor {i+1} Name'), sg.InputText(key=f'factor_name_{i}'),
         sg.Text('Impact on (C for Cost or T for Contract)'), sg.InputText(key=f'impact_on_{i}'),
         sg.Text('Impact Value'), sg.InputText(key=f'impact_value_{i}')] for i in range(num_factors)
    ]
    layout_factors.append([sg.Button('Calculate'), sg.Button('Exit')])

    window_factors = sg.Window('Factor Information').Layout(layout_factors)

    while True:
        event, values = window_factors.Read()
        if event == 'Calculate':
            factors = {}
            for i in range(num_factors):
                factor_name = values.get(f'factor_name_{i}', '')  # Using get method to handle missing keys
                impact_on = values.get(f'impact_on_{i}', '')
                impact_value = float(values.get(f'impact_value_{i}', 0))
                factors[factor_name] = (impact_on, impact_value)

            original_contract, original_cost = contract, cost  # Store original values
            contract_val, cost_val, possibilities = calculate_profit_percentage(original_contract, original_cost, factors)
            generate_pdf(original_contract, original_cost, possibilities)
            break
        if event == 'Exit' or event is None:
            break

    window_factors.Close()

if __name__ == '__main__':
    main()
